In [81]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [19]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [77]:
def parser(x):
	return datetime.strptime(x+'20', '%m/%d/%Y')

In [79]:
countries = ['Austria', 'Brazil', 'China', 'Czechia', 'Denmark', 'France', 'Germany', 'India', 'Italy', 'Korea_South', 'South_Africa', 'Spain', 'UK']
# countries = ['Austria', 'Brazil', 'China', 'Czechia', 'Denmark', 'France', 'Germany', 'India', 'Italy', 'Korea_South', 'Spain', 'UK']
results = []
for country in countries:
    result = {'country':'', 'conf': 0, 'dead': 0, 'recov': 0}
    result['country'] = country
    file = '../Data_Preprocessing/Clean/' + country + '.csv'
    df = pd.read_csv(file, index_col = ['Date'], parse_dates = ['Date'], date_parser = parser, squeeze = True)
    for index, row in df.iterrows():
        df.at[index, 'Confirmed_Cases'] = float(row['Confirmed_Cases'])
        df.at[index, 'Recoveries'] = float(row['Recoveries'])
        df.at[index, 'Deaths'] = float(row['Deaths'])
        
    confirmed = df[['Confirmed_Cases']]
    recoveries = df[['Recoveries']]
    deaths = df[['Deaths']]
    
    new_header = confirmed.iloc[0]
    confirmed = confirmed[1:]
    confirmed.columns = new_header
    
    new_header = recoveries.iloc[0]
    recoveries = recoveries[1:]
    recoveries.columns = new_header
    
    new_header = deaths.iloc[0]
    deaths = deaths[1:]
    deaths.columns = new_header
    
    confirmed = confirmed.values
    deaths = deaths.values
    recoveries = recoveries.values
    
    sp = int(len(confirmed)*0.8)
    
    X, y = train_test_split(confirmed, test_size=0.2, random_state=42)
    X_r, y_r = train_test_split(recoveries, test_size=0.2, random_state=42)
    X_d, y_d = train_test_split(deaths, test_size=0.2, random_state=42)
    
#     X, y = confirmed[0:sp], confirmed[sp:]
#     X_r, y_r = recoveries[0:sp], recoveries[sp:]
#     X_d, y_d = deaths[0:sp], deaths[sp:]
    
    history = [a for a in X]
    predictions = []
    
    for t in range(len(y)):
        model = ARIMA(history, order=(1, 0, 0))
        fit = model.fit(disp = 0)
        output = fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        history.append(y[t])
    error = mean_squared_error(y, predictions)
    result['conf'] = error
    
    history = [a for a in X_d]
    predictions = []
    
    for t in range(len(y_d)):
        model = ARIMA(history, order=(1, 1, 0))
        fit = model.fit(disp = 0)
        output = fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        history.append(y_d[t])
    error = mean_squared_error(y_d, predictions)
    result['dead'] = error
    
    history = [a for a in X_r]
    predictions = []
    
    for t in range(len(y_r)):
        model = ARIMA(history, order=(1, 0, 0))
        fit = model.fit(disp = 0)
        output = fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        history.append(y_r[t])
    error = mean_squared_error(y_r, predictions)
    result['recov'] = error
    
    results.append(result)
    
results

D:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\venv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  


NameError: name 'train_test_split' is not defined